In [1]:
import mysql.connector
import json, os, re
from datetime import datetime
import pandas as pd
import re
base_dir = os.path.join(os.path.dirname(os.getcwd())+r"\Kaustubh")

In [3]:
def get_json_paths(root_folder):
    json_paths = []
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isfile(folder_path) and folder_path.endswith(".json"):
            json_paths.append(folder_path)
    return json_paths

def load_all_records(json_paths):
    all_records ={}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                records = data.get("records", [])
                name = " ".join(records[0].get("value",{}).get("amc_name","").split(" ")[:2])
                # print(" ".join(name[:2]))
                if name not in all_records:
                    all_records[name] = records
                else:
                    name+=" 2"
                    all_records[name] = records
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)

# '360', 'Aditya', 'Bajaj', 'Bandhan', 'Bank', 'Baroda', 'Canara', 'Dsp', 'Edelweiss', 'Groww', 'Hdfc', 'Helios', 'Invesco', 'Iti', 'Kotak', 'Lic', 
# 'Mahindra', 'Mirae', 'Navi', 'Nippon', 'Nj', 'Old', 'Pgim', 'Quantum', 'Quant', 'Samco', 'Sbi', 'Sundaram', 'Tata', 'Taurus', 'Trustmf', 'Whiteoak', 'Zerodha'

# print(json.dumps(combined_records[0], indent=2))


Error reading c:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\Kaustubh\41_31-Jan-25_FS.json: Extra data: line 643 column 8 (char 15502)


In [ ]:
# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306
# )
# cursor = conn.cursor()
# print("Successful Connection !!")
# conn = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="1234",
#     database="cog_updated_db"
# )

cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM mf_amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO mf_amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

def insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month):
    mutual_fund_keys = ["amc_id", "entered_time", "amc_for_month", "amc_name", "benchmark_index", "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", 
                        "monthly_aaum_value", "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]
    
    mutual_fund_query = f"INSERT INTO mf_mutual_funds ({', '.join(mutual_fund_keys)}) VALUES ({', '.join(['%s'] * len(mutual_fund_keys))})"
    values = [amc_id, curr_time, amc_month] + [details.get(key, "") for key in mutual_fund_keys[3:]]
    cursor.execute(mutual_fund_query, values)
    return cursor.lastrowid  # return mutual_fund_id

def insert_fund_managers(cursor, amc_id, mutual_fund_id, details):
    fund_manager_keys = ["amc_id", "mutual_fund_id", "main_scheme_name", "name", "qualification", "managing_fund_since", "total_exp"]
    fund_manager_query = f"INSERT INTO mf_fund_managers ({', '.join(fund_manager_keys)}) VALUES ({', '.join(['%s'] * len(fund_manager_keys))})"
    if "fund_manager" in details and isinstance(details['fund_manager'],list):
        for manager in details.get("fund_manager", []):
            values = [amc_id, mutual_fund_id, details["main_scheme_name"]] + [manager.get(col, '') for col in fund_manager_keys[3:]]
            cursor.execute(fund_manager_query, values)
    else:
        print(f"Error in Managers, {details['main_scheme_name']}")

def insert_load(cursor, amc_id, mutual_fund_id, details):
    load_keys = ['amc_id', 'mutual_fund_id', 'main_scheme_name', 'entry_load', 'exit_load']
    if "load" in details and isinstance(details["load"],dict):
        load_query = f"INSERT INTO mf_transformed_loads ({', '.join(load_keys)}) VALUES ({', '.join(['%s'] * len(load_keys))})"
        values = [amc_id, mutual_fund_id, details["main_scheme_name"]] + [details.get("load", {}).get(col, '') for col in load_keys[3:]]
        cursor.execute(load_query, values)
    else:
        print(f"Error in Loads, {details['main_scheme_name']}")

def insert_metrics(cursor, amc_id, mutual_fund_id, details):
    metric_keys = ['amc_id', 'main_scheme_name', 'mutual_fund_id', "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity", 
                   "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay", "mod_duration", "port_turnover_ratio", "r_squared_ratio", 
                   "roe_ratio", "sharpe", "sortino_ratio", "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"]
    if "metrics" in details and isinstance(details["metrics"],dict):
        metric_query = f"INSERT INTO mf_transformed_metrics ({', '.join(metric_keys)}) VALUES ({', '.join(['%s'] * len(metric_keys))})"
        values = [amc_id, details["main_scheme_name"], mutual_fund_id] + [details.get("metrics", {}).get(col, '') for col in metric_keys[3:]]
        cursor.execute(metric_query, values)
    else:
        print(f"Error in Metrics, {details['main_scheme_name']}")
        

for scheme_name, details in data.items():
    amc_id = get_or_create_amc(details["amc_name"])
    curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
    amc_month = "JAN 25"

    mutual_fund_id = insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month)
    insert_fund_managers(cursor, amc_id, mutual_fund_id, details)
    insert_load(cursor, amc_id, mutual_fund_id, details)
    insert_metrics(cursor, amc_id, mutual_fund_id, details)

conn.commit()
cursor.close()
conn.close()
print("Working !!")

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="finkstein_db"
)
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid

def insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month):
    keys = ["amc_id", "entered_time", "amc_for_month", "amc_name", "benchmark_index",
            "main_scheme_name", "mutual_fund_name", "monthly_aaum_date", "monthly_aaum_value",
            "scheme_launch_date", "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple"]
    
    query = f"""
        INSERT INTO mutual_funds ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    values = [
        amc_id, curr_time, amc_month, details.get("amc_name", ""),
        ", ".join(details.get("benchmark_index", [])),
        details.get("main_scheme_name", ""),
        details.get("mutual_fund_name", ""),
        details.get("monthly_aaum_date", ""),
        details.get("monthly_aaum_value", ""),
        details.get("scheme_launch_date", ""),
        details.get("min_addl_amt", ""),
        details.get("min_addl_amt_multiple", ""),
        details.get("min_amt", ""),
        details.get("min_amt_multiple", "")
    ]

    cursor.execute(query, values)
    return cursor.lastrowid

def insert_fund_managers(cursor, amc_id, mutual_fund_id, details):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO fund_managers
            (amc_id, mutual_fund_id, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            amc_id,
            mutual_fund_id,
            details["main_scheme_name"],
            manager.get("name", ""),
            manager.get("qualification", ""),
            manager.get("managing_fund_since", ""),
            manager.get("total_exp", "")
        ]
        cursor.execute(query, values)

def insert_load(cursor, amc_id, mutual_fund_id, details):
    entry_load = exit_load = ""
    for item in details.get("load", []):
        if item.get("type") == "entry":
            entry_load = item.get("comment", "")
        elif item.get("type") == "exit":
            exit_load = item.get("comment", "")

    query = """
        INSERT INTO transformed_loads
        (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s, %s, %s, %s, %s)
    """
    values = [
        amc_id,
        mutual_fund_id,
        details.get("main_scheme_name", ""),
        entry_load,
        exit_load
    ]
    cursor.execute(query, values)

def insert_metrics(cursor, amc_id, mutual_fund_id, details):
    metric_template = {
        "alpha": "", "arithmetic_mean_ratio": "", "average_div_yld": "", "average_pb": "", "average_pe": "",
        "avg_maturity": "", "beta": "", "correlation_ratio": "", "downside_deviation": "", "information_ratio": "",
        "macaulay": "", "mod_duration": "", "port_turnover_ratio": "", "r_squared_ratio": "", "roe_ratio": "",
        "sharpe": "", "sortino_ratio": "", "std_dev": "", "tracking_error": "", "treynor_ratio": "",
        "upside_deviation": "", "ytm": ""
    }

    for metric in details.get("metrics", []):
        metric_name = metric.get("name")
        if metric_name in metric_template:
            metric_template[metric_name] = metric.get("value", "")

    keys = ["amc_id", "main_scheme_name", "mutual_fund_id"] + list(metric_template.keys())
    query = f"""
        INSERT INTO transformed_metrics
        ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """
    values = [
        amc_id,
        details.get("main_scheme_name", ""),
        mutual_fund_id
    ] + list(metric_template.values())

    cursor.execute(query, values)

# Begin inserting all records
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "JAN 25"


for amc_key, records in combined_records.items():
    print(f"Doing {amc_key}")
    for record in records:
        details = record["value"]
        amc_id = get_or_create_amc(details["amc_name"])
        mutual_fund_id = insert_mutual_fund(cursor, amc_id, details, curr_time, amc_month)
        insert_fund_managers(cursor, amc_id, mutual_fund_id, details)
        insert_load(cursor, amc_id, mutual_fund_id, details)
        insert_metrics(cursor, amc_id, mutual_fund_id, details)

conn.commit()
cursor.close()
conn.close()
print("Data insertion completed.")


In [5]:
def sanitize_sheet_name(name):
    name = re.sub(r'[^A-Za-z0-9_ ]+', '', name)
    name = name.strip().replace(" ", "_")
    return name[:31]

def fetch_data_from_db(host, user, password, database, port, amc_table, mf_table, load_table, metric_table):
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=port
    )

    amcs_df = pd.read_sql(f"SELECT amc_name FROM {amc_table};", conn)
    amc_names = amcs_df['amc_name'].tolist()

    query_template = f"""
    SELECT 
        mf.*, tl.entry_load, tl.exit_load,
        tm.alpha, tm.arithmetic_mean_ratio, tm.average_div_yld, tm.average_pb, tm.average_pe,
        tm.avg_maturity, tm.beta, tm.correlation_ratio, tm.downside_deviation,
        tm.information_ratio, tm.macaulay, tm.mod_duration, tm.port_turnover_ratio,
        tm.r_squared_ratio, tm.roe_ratio, tm.sharpe,
        tm.sortino_ratio, tm.std_dev, tm.tracking_error, tm.treynor_ratio,
        tm.upside_deviation, tm.ytm
    FROM {mf_table} mf
    LEFT JOIN {load_table} tl ON mf.id = tl.mutual_fund_id
    LEFT JOIN {metric_table} tm ON mf.id = tm.mutual_fund_id
    WHERE mf.amc_name LIKE %s;
    """

    data_by_amc = {}
    cursor = conn.cursor(dictionary=True)

    for amc_name in amc_names:
        like_pattern = f"%{amc_name}%"
        cursor.execute(query_template, (like_pattern,))
        rows = cursor.fetchall()
        if rows:
            df = pd.DataFrame(rows)
            columns_to_drop = ["id", "amc_id", "entered_time", "amc_for_month", "amc_name"]
            df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
            if "main_scheme_name" in df.columns:
                cols = ["type", "main_scheme_name"] + [col for col in df.columns if col not in ["main_scheme_name"]]
            else:
                cols = ["type"] + df.columns.tolist()
            df.insert(0, "type", database)
            df = df[cols]
            data_by_amc[amc_name] = df

    cursor.close()
    conn.close()
    return data_by_amc

finkstein_data = fetch_data_from_db(
    host="localhost",
    user="root",
    password="1234",
    database="finkstein_db",
    port=3306,
    amc_table="amcs",
    mf_table="mutual_funds",
    load_table="transformed_loads",
    metric_table="transformed_metrics"
)

mydata_data = fetch_data_from_db(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    amc_table="mf_amcs",
    mf_table="mf_mutual_funds",
    load_table="mf_transformed_loads",
    metric_table="mf_transformed_metrics"
)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
finkstein_path = f"finkstein_export_{timestamp}.xlsx"
mydata_path = f"mydata_export_{timestamp}.xlsx"

with pd.ExcelWriter(finkstein_path, engine="xlsxwriter") as writer1:
    for amc_name, df in finkstein_data.items():
        sheet_name = sanitize_sheet_name(amc_name)
        df.to_excel(writer1, sheet_name=sheet_name, index=False)

with pd.ExcelWriter(mydata_path, engine="xlsxwriter") as writer2:
    for amc_name, df in mydata_data.items():
        sheet_name = sanitize_sheet_name(amc_name)
        df.to_excel(writer2, sheet_name=sheet_name, index=False)

print("Data exported to Excel successfully!")
print("Files created:")
print(f" - {finkstein_path}")
print(f" - {mydata_path}")


C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_11944\2271205200.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  amcs_df = pd.read_sql(f"SELECT amc_name FROM {amc_table};", conn)
C:\Users\Kaustubh.keny\AppData\Local\Temp\ipykernel_11944\2271205200.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  amcs_df = pd.read_sql(f"SELECT amc_name FROM {amc_table};", conn)


Data exported to Excel successfully!
Files created:
 - finkstein_export_20250401_1244.xlsx
 - mydata_export_20250401_1244.xlsx
